In [1]:
import sys
!{sys.executable} -m pip install geocoder

In [2]:
import requests
import pandas as pd
import lxml.html as lh
import geocoder

print("Libraries imported!")

Libraries imported!


In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website
webcontent = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = webcontent.xpath('//tr')

In [4]:
#Create empty lists
postal_code_list=[]
borough_list=[]
neighborhood_list=[]

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for idx in range(1,len(tr_elements)):
    
    tr_element = tr_elements[idx]
    
    #bypass if no.of column not equal to 3
    if len(tr_element)!=3:
        break
    
    col_idx=0
    postal_code=''
    borough=''
    neighborhood=''
    for elem in tr_element.iterchildren():
        data=elem.text_content()
        data=data.replace('\n','')
        
        if col_idx == 0:
            postal_code = data
        elif col_idx == 1:
            borough = data
        elif col_idx == 2:
            neighborhood = data

        col_idx+=1
        
    #only add to list if postalcode starts with 'M' and Borough not equal to 'Not assigned'
    if postal_code.startswith('M') and borough != 'Not assigned':
        postal_code_list.append(postal_code)
        borough_list.append(borough)
        neighborhood_list.append(neighborhood)

print('Length of PostalCode {}'.format(len(postal_code_list)))
print('Length of Borough {}'.format(len(borough_list)))
print('Length of Neghoborhood {}'.format(len(neighborhood_list)))

Length of PostalCode 103
Length of Borough 103
Length of Neghoborhood 103


In [6]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

df = pd.DataFrame(columns=column_names)

df

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [7]:
# initialize your variable to None
lat_lng_coords = None

for idx in range(0, len(postal_code_list)):
    postal_code = postal_code_list[idx]
    borough = borough_list[idx]
    neighborhood = neighborhood_list[idx]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

    df = df.append({'PostalCode': postal_code,
                    'Borough': borough,
                    'Neighborhood': neighborhood,
                    'Latitude': lat_lng_coords[0],
                    'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [8]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


In [9]:
df.tail()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653340,-79.509766
99,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.648700,-79.385450
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632798,-79.493017
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.625490,-79.526000


In [10]:
df.shape

(103, 5)